## 식자재 최저가 검색 서비스 

1. 목적
    - 식자재 구매 시 판매 업체별 가격비교가 어려움
    - 네이버 최저가 서비스에서 검색되지 않는 실제 최저가를 탐색하기 어려움

    - **-> 온라인 식자재몰, 일반 소매 업체몰 등 통합적 가격비교 서비스 제공**


2. 실행 계획

    1. 1차적으로 각 온라인몰의 사이트를 1개씩 정하여 크롤링 
        - 배민상회, 이마트(오픈몰), 다담몰(식자재몰) ... \
          (각 분야별 사이트는 추가될 예정)
    2. 컬럼은 category, title, min_gram, price_per_gram, link 로 구성
    3. 각 사이트별 크롤링에서 직면한 어려움들.. *개별 상세 질문예정*
    4. 3개의 사이트 완료 후 가능하다면 다른 사이트도 추가할 예정
    5. `향후 계획` 일정 주기별 크롤링 후 최저가 갱신 시 slack, 카톡 등 알림전송 서비스
    
|사이트 선정| 크롤링 test(local) | SQL DB에 데이터 저장 | 알림전송|
|:-----------:|:---------------:|:----------------------:|:---------:|
|배민상회| 확인 | 확인(질문) | - |
|이마트| 확인중(질문) | - | - |
|다담몰| 확인중(질문) | - | _ |


> ### 배민상회 크롤링 test(local)

`url = "https://mart.baemin.com/goods/list/{}".format(category)`

```
category = {
    "100015" : "냉동상품",
    "100024" : "냉장식품",
    "100028" : "유제품",
    "100032" : "식용유/설탕/소금",
    "100038" : "장류/조미료",
    "100043" : "소스/양념/육수",
    "100050" : "밀가루/라면/믹스류",
    "100054" : "편의식/가공기타",
    "100061" : "반찬류",
}
```

- 시도해보기 

In [ ]:
# 전체 카테고리 중 가공식품-냉동식품-전체 크롤링 테스트 

In [1]:
url = "https://mart.baemin.com/api/v2/front/categories/goods/100015/paging?goodsSortType=BASIC&page=0"
response = requests.get(url)
response

<Response [200]>

In [2]:
data = response.json()['data']['simpleGoodsDtoPage']['content']
data

[{'id': 14989,
  'name': '[든든] 닭목살튀김 마일드 500g 4개 ',
  'listImage': 'https://cdn-mart.baemin.com/sellergoods/list/b4ad22ae-47b4-49a0-9df4-528c05654e40.jpg',
  'goodsPrice': 38800,
  'customerPrice': 43200,
  'discountRate': 10,
  'discountAmount': 4400,
  'discountType': 'DISCOUNT_RATE',
  'freshShipping': False,
  'newGoods': True,
  'open': True,
  'salesStartDate': '2020-09-10T04:27:00.000+00:00',
  'display': True,
  'soldOut': False,
  'sale': True,
  'sizeDesc': '500g',
  'priceDesc': '100g 당 1,940원 / 4ea',
  'memberShip': False,
  'applySample': False,
  'customMade': False,
  'tags': ['트랜드', '안주', '사이드메뉴'],
  'icePackAvailability': False},
 {'id': 14988,
  'name': '[든든] 닭목살튀김 매운맛 500g 4개 ',
  'listImage': 'https://cdn-mart.baemin.com/sellergoods/list/0cae5d6c-5a4c-461c-98a9-01e1a96ad117.jpg',
  'goodsPrice': 38800,
  'customerPrice': 43200,
  'discountRate': 10,
  'discountAmount': 4400,
  'discountType': 'DISCOUNT_RATE',
  'freshShipping': False,
  'newGoods': True,
  'open': T

In [3]:
data = response.json()['data']['simpleGoodsDtoPage']['content'][0]
data['name'],data['goodsPrice'],data['customerPrice'],data['sizeDesc'],data['priceDesc'],data['id']

('[든든] 닭목살튀김 마일드 500g 4개 ', 38800, 43200, '500g', '100g 당 1,940원 / 4ea', 14989)

In [4]:
# 한번에 크롤링 되는지 다름 상품군 (전체카테고리-가공식품-반찬류) 테스트

In [5]:
url = "https://mart.baemin.com/api/v2/front/categories/goods/100061/paging?goodsSortType=BASIC&page=1"
response = requests.get(url)
elements = response.json()['data']['simpleGoodsDtoPage']['content']
element = response.json()['data']['simpleGoodsDtoPage']['content'][0]
element['name'],element['goodsPrice'],element['customerPrice'],element['sizeDesc'],element['priceDesc'], element['id']

('[든든]새찬 프레시오이피클 3kg', 5810, 5810, '3kg', '개당 5,810원 / 1ea', 8592)

In [6]:
# 만약에 없는 페이지로 요청할 경우에는 []으로 표시되어 [0]에서 에러발생.

In [7]:
url = "https://mart.baemin.com/api/v2/front/categories/goods/100061/paging?goodsSortType=BASIC&page=6"
response = requests.get(url)
elements = response.json()['data']['simpleGoodsDtoPage']['content']
elements
# element = response.json()['data']['simpleGoodsDtoPage']['content'][0]
# element['name'],element['goodsPrice'],element['customerPrice'],element['sizeDesc'],element['priceDesc']

[]

In [8]:
url = "https://mart.baemin.com/api/v2/front/categories/goods/100050/paging?goodsSortType=BASIC&page=0"
response = requests.get(url)
elements = response.json()['data']['simpleGoodsDtoPage']['content']
# elements
element = response.json()['data']['simpleGoodsDtoPage']['content'][0]
element['name'],element['goodsPrice'],element['customerPrice'],element['sizeDesc'],element['priceDesc'], element['id']

('[든든] 팔도비빔면 130g 40개', 25500, 25500, '130g', '개당 638원 / 40ea', 14811)

In [9]:
# 함수로 만들기 도전..!
# 우선 가공식품 먼저 

In [10]:
categories = {
    100015 : "냉동상품",
    100024 : "냉장식품",
    100028 : "유제품",
    100032 : "식용유/설탕/소금",
    100038 : "장류/조미료",
    100043 : "소스/양념/육수",
    100050 : "밀가루/라면/믹스류",
    100054 : "편의식/가공기타",
    100061 : "반찬류",
}

In [11]:
def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice'] -> 원가는 표시하지 않기로 함.
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])
        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

In [12]:
baemin_items(100015, 0)

,category,title,price,min_gram,price_per_gram,link
0,냉동상품,[든든] 닭목살튀김 마일드 500g 4개,38800,500g,"100g 당 1,940원 / 4ea",https://mart.baemin.com/goods/detail/14989
1,냉동상품,[든든] 닭목살튀김 매운맛 500g 4개,38800,500g,"100g 당 1,940원 / 4ea",https://mart.baemin.com/goods/detail/14988
2,냉동상품,[든든] 김밥용 스모크 소시지 400g,3900,400g,100g 당 975원 / 1ea,https://mart.baemin.com/goods/detail/14977
3,냉동상품,[든든] 훈제 삼겹 슬라이스 1kg,19400,1kg,"1kg 당 19,400원 / 1ea",https://mart.baemin.com/goods/detail/14976
4,냉동상품,[든든] 모듬소시지3 410g,5600,410g,"100g 당 1,366원 / 1ea",https://mart.baemin.com/goods/detail/14974
...,...,...,...,...,...,...
95,냉동상품,[든든] 호키생선까스 (80g 20입) 1.6kg,14000,1.6kg(80g*20ea),"개당 14,000원 / 1ea",https://mart.baemin.com/goods/detail/11251
96,냉동상품,[든든] 하림이닭 소시지양파 120g [13시 마감],1700,120g,"개당 1,700원 / 1ea",https://mart.baemin.com/goods/detail/11033
97,냉동상품,[든든] 새우품은까스 (55g 24입) 1.32kg,18400,1.32kg(55g*24ea),"개당 18,400원 / 1ea",https://mart.baemin.com/goods/detail/11250
98,냉동상품,[든든] 하림이닭 소시지청양고추 120g [13시 마감],1700,120g,"개당 1,700원 / 1ea",https://mart.baemin.com/goods/detail/11032


In [13]:
baemin_items(100015, 11)

""


In [14]:
categories

{100015: '냉동상품',
 100024: '냉장식품',
 100028: '유제품',
 100032: '식용유/설탕/소금',
 100038: '장류/조미료',
 100043: '소스/양념/육수',
 100050: '밀가루/라면/믹스류',
 100054: '편의식/가공기타',
 100061: '반찬류'}

In [15]:
dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

냉동상품
0 1 2 3 4 5 6 7 8 9 10 11 
냉장식품
0 1 2 3 4 5 6 7 8 9 10 11 
유제품
0 1 2 3 4 5 6 7 8 9 10 11 
식용유/설탕/소금
0 1 2 3 4 5 6 7 8 9 10 11 
장류/조미료
0 1 2 3 4 5 6 7 8 9 10 11 
소스/양념/육수
0 1 2 3 4 5 6 7 8 9 10 11 
밀가루/라면/믹스류
0 1 2 3 4 5 6 7 8 9 10 11 
편의식/가공기타
0 1 2 3 4 5 6 7 8 9 10 11 
반찬류
0 1 2 3 4 5 6 7 8 9 10 11 


In [19]:
df_1 = pd.concat(dfs, ignore_index=True)
# df_1["title"] 
df_1

,category,title,price,min_gram,price_per_gram,link
0,냉동상품,[든든] 닭목살튀김 마일드 500g 4개,38800,500g,"100g 당 1,940원 / 4ea",https://mart.baemin.com/goods/detail/14989
1,냉동상품,[든든] 닭목살튀김 매운맛 500g 4개,38800,500g,"100g 당 1,940원 / 4ea",https://mart.baemin.com/goods/detail/14988
2,냉동상품,[든든] 김밥용 스모크 소시지 400g,3900,400g,100g 당 975원 / 1ea,https://mart.baemin.com/goods/detail/14977
3,냉동상품,[든든] 훈제 삼겹 슬라이스 1kg,19400,1kg,"1kg 당 19,400원 / 1ea",https://mart.baemin.com/goods/detail/14976
4,냉동상품,[든든] 모듬소시지3 410g,5600,410g,"100g 당 1,366원 / 1ea",https://mart.baemin.com/goods/detail/14974
...,...,...,...,...,...,...
3105,반찬류,리치스 컵 오이피클 80g 168개 [13시마감],29900,80g,100g 당 223원 / 168ea,https://mart.baemin.com/goods/detail/4528
3106,반찬류,델솔 할라피뇨 슬라이스 2.83kg 6개 [13시마감],41000,2.83kg,"개당 6,834원 / 6ea",https://mart.baemin.com/goods/detail/4521
3107,반찬류,리치스 오이피클 슬라이스 3kg 6개 [13시마감],32900,3kg,"1kg 당 1,828원 / 6ea",https://mart.baemin.com/goods/detail/3999
3108,반찬류,코포리바 블랙 올리브 슬라이스 1.8kg 10개 [13시마감],41000,1.8kg,"1kg 당 2,278원 / 10ea",https://mart.baemin.com/goods/detail/3990


In [20]:
%%writefile baemin_mart_items.py

import pandas as pd
import requests 
import time

categories = {
    100015 : "냉동상품",
    100024 : "냉장식품",
    100028 : "유제품",
    100032 : "식용유/설탕/소금",
    100038 : "장류/조미료",
    100043 : "소스/양념/육수",
    100050 : "밀가루/라면/믹스류",
    100054 : "편의식/가공기타",
    100061 : "반찬류",
    100067 : "야채/채소",
    100077 : "과일",
    100081 : "쌀/잡곡/견과",
    100085 : "수산/건어물",
    100096 : "축산/계란",
}

def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])
        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
        
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

# global gagong_df -> 파이썬 파일을 실행 했을 때 gagong_df가 나왔으면 좋겠음.. 
baemin_mart_items_df = pd.concat(dfs, ignore_index=True)
baemin_mart_items_df

baemin_mart_items.to_csv("baemin_mart_items.csv", index=False, encoding="utf-8-sig")

print("="*100)
print("It's saved as baemin_mart_items.csv file.")
print("="*100)

Overwriting baemin_mart_items.py


In [21]:
%%writefile baemin_mart_delivery_goods.py

import pandas as pd
import requests 
import time

# categories = {
#     100104 : "플라스틱용기",
#     100105 : "종이용기/박스",
#     100138 : "카페/음료용기",
#     100159 : "종이/비닐봉투",
#     100165 : "수저/위생용품",
#     100177 : "스티커/배너/포스터",
#     100184 : "포장재",
#     100188 : "기타가게용품",
# }

categories = {
    100104 : "포장용품",
    100105 : "포장용품",
    100138 : "포장용품",
    100159 : "포장용품",
    100165 : "가게위생용품",
    100184 : "포장용품",
    100188 : "가게위생용품",
}


def baemin_items(category, page):
    url = "https://mart.baemin.com/api/v2/front/categories/goods/{}/paging?goodsSortType=BASIC&page={}".format(category, page)
    response = requests.get(url)
    elements = response.json()['data']['simpleGoodsDtoPage']['content']
    datas = []
    for element in elements:
        title = element['name']
        price = element['goodsPrice']
#         o_price = element['customerPrice']
        # 원인은 모르겠지만.. 100050/밀가루 쪽에서 에러남
        try :
            min_gram = element['sizeDesc']
        except:
            min_gram = "-"
        try:
            price_per_gram = element['priceDesc']
        except:
            price_per_gram = "-"
        link = "https://mart.baemin.com/goods/detail/" + str(element['id'])
        datas.append({
            "category": categories[category],
            "title" : title,
            "price" : price,
#             "o_price" : o_price,
            "min_gram" : min_gram,
            "price_per_gram" : price_per_gram,
            "link" : link,
        })
    return pd.DataFrame(datas)

dfs = []
for category in categories:
    print(categories[category])
    for page in range(12): # url에서 page가 0부터 시작
        print(page, end=" ")
        df = baemin_items(category, page)
        dfs.append(df)
    print()
    time.sleep(2)

# global gagong_df -> 파이썬 파일을 실행 했을 때 gagong_df가 나왔으면 좋겠음.. 
baemin_mart_delivery_goods_df = pd.concat(dfs, ignore_index=True)
baemin_mart_delivery_goods_df

baemin_mart_delivery_goods_df.to_csv("baemin_mart_delivery_goods.csv", index=False, encoding="utf-8-sig")

print("="*100)
print("It's saved as baemin_mart_delivery_goods.csv file.")
print("="*100)

Overwriting baemin_mart_delivery_goods.py


In [ ]:
# 크롤링 날짜를 넣어서 시기별 비교는 어떨까...??

In [12]:
from datetime import datetime

In [14]:
datetime.now()

datetime.datetime(2020, 9, 9, 22, 25, 24, 867358)

In [18]:
datetime.now().strftime('%Y-%m-%d')

'2020-09-09'

# 우선 sql에 넣어보자

> **이후 과정은 서버에서 진행함\
baemin_MySQLdb.ipynb, baemin_sqlalchemy.ipynb 참고**

In [2]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import MySQLdb

In [3]:
import configparser

In [4]:
config = configparser.ConfigParser()
config.read("../db.ini")
sql_datas = config["sql"]

In [5]:
engine = create_engine("mysql://{}:{}@{}/test".format(sql_datas['user'], sql_datas['pw'], sql_datas['public_ip']))

In [ ]:
# # 테이블 객체 생성을 위한 클래스 작성
# Base = declarative_base()

# class User(Base):
    
#     __tablename__ = "user" # 테이블 이름
    
#     # 컬럼 데이터 작성
#     user_id = Column(Integer, primary_key=True)
#     name = Column(String(20))
#     email = Column(String(30))
#     age = Column(Integer)
#     rdate = Column(DateTime)
    
#     # 생성자 함수
#     def __init__(self, name, email, age, rdate):
#         self.name = name
#         self.email = email
#         self.age = age
#         self.rdate = rdate
        
#     # repr 함수
#     def __repr__(self):
#         return "<User {}, {}, {}, {}>".format(
#             self.name, self.email, self.age, self.rdate)
           

## 스크래피 도전했으나 실패... 다시 여쭤보고 도전하기

In [ ]:
# import scrapy
# import requests
# from scrapy.http import TextResponse
# from fake_useragent import UserAgent

In [ ]:
# 1. 프로젝트 생성

In [28]:
# !scrapy startproject baemin_test

New Scrapy project 'baemin_test', using template directory 'c:\users\82106\anaconda3\envs\fc14th\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\82106\Desktop\dss14th\project\3_crawling_4\baemin_test

You can start your first spider with:
    cd baemin_test
    scrapy genspider example example.com


In [29]:
# 2. xpath 확인

In [ ]:
# link

In [55]:
# UserAgent().chrome

'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'

In [78]:
headers = {
    'User-Agent' : UserAgent().chrome,
    'referer': 'https://mart.baemin.com/'
}

In [79]:
# url = "https://mart.baemin.com/goods/list/100015"
# req = requests.get(url, headers=headers)
# response = TextResponse(req.url, body=req.text, encoding='utf-8')
# response

<200 https://mart.baemin.com/goods/list/100015>

In [80]:
# req.text

'<!DOCTYPE html><html lang="ko"><head><title>상품목록 - 배민상회</title><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"><meta name="description" content="신선 식재료, 포장용기, 배달비품 전문 쇼핑몰, 배민상회입니다. 톡톡튀는 배민 디자인 제품과 품질로 엄선한 기성품을 특별 할인가로 만나보세요."><meta name="keywords" content="식재료, 가공 식재료, 닭고기, 소스, 플라스틱 용기, 비닐, 수저, 실링용기, 치킨박스, 피자박스, 도시락/반찬용기"><meta name="naver-site-verification" content="5b135fdbe175231fdfc9b895e54dfaab67a0440d"><meta name="google-site-verification" content="d-63bvk94V8zeqX9pD9T0ZzJIScTPHelZrjFwjYzEeU"><meta property="og:site_name" content="배민상회"><meta property="og:title" content="상품목록 - 배민상회"><meta property="og:type" content="website"><meta property="og:description" content="신선 식재료, 포장용기, 배달비품 전문 쇼핑몰, 배민상회입니다. 톡톡튀는 배민 디자인 제품과 품질로 엄선한 기성품을 특별 할인가로 만나보세요."><meta property="og:url" content="https://mart.baemin.com"><meta property="og:image" content="https://cdn-mart.baemin.com/front-end/assets-static/og-image-logo.jpg"><script src="/cdn-cgi/apps/head/SYqbcWF5TY82iE

In [82]:
# links = response.xpath('//*[@id="root"]/div/div[2]/div[2]/section/div/div/a').extract()
# links

[]

In [83]:
# title = response.xpath('//*[@id="root"]/div/div[2]/div[2]/section/div[1]/div/a/div[2]/div/p[1]/text()')
# title

[]

## 여기까지 스크래피 도전기